In [2]:
import pandas as pd
import seaborn as sns; sns.set()
pd.options.display.max_columns=100
pd.options.display.max_rows=300
import numpy as np

### z플립4% 서치클릭코어커런스 데이터 불러오기 및 전처리
- 오피셜 런칭 이후 2 주치 데이터: 2022.8-9

In [ ]:
"""데이터 불러오기"""
data_co=pd.read_csv('/home/ncp/workspace/nasw2/20231129_22y2h_cc_%z플립%.csv',
                    names=['age','loc1','loc2','keyword1','area1','keyword2','area2','count','date','device','gender'])

In [ ]:
data_co.head()

In [ ]:
"""데이터 전처리1"""
data_co.reset_index(inplace=True)
data_co=data_co.iloc[:,1:-1]                # 필요 없는 열 삭제
data_co.columns=list(data_co.iloc[0,:])     # 컬럼 설정
data_co=data_co.iloc[1:,:]                  # 첫번째 행 삭제
data_co.reset_index(drop=True, inplace=True)

In [ ]:
data_co.head()

In [ ]:
data_co.columns

In [ ]:
"""데이터 전처리2"""
data_co.columns = [str(col).strip() for col in data_co.columns] # 컬럼명 앞뒤 공백 삭제
for col in data_co.columns:
    data_co[col] = data_co[col].apply(lambda x: x.strip() if isinstance(x, str) else x) # 내용 앞뒤 공백 삭제
for col in data_co.columns:
    data_co.drop(data_co[data_co[col]==col].index, inplace=True)    # 행 내 컬럼명과 동일한 행 삭제
data_co.columns = [col.split('.')[1] for col in data_co.columns]    # 컬럼명에 추가된 데이터명 삭제
data_co['date'] = pd.to_datetime(data_co['date'], errors='coerce')  # 날짜 형식 변경
data_co['count'] = data_co['count'].apply(lambda x: int(x) if isinstance(x, str) and x.isdigit() else np.nan)   # count 변수 (object::int)

In [ ]:
data_co.columns

In [ ]:
print(len(data_co['date'].value_counts()), '일치 데이터', sep='')   # 2달치 데이터 2023.07-08
print(format(len(data_co), ','), '행 데이터', sep='')

In [ ]:
temp = data_co.loc[~data_co['keyword2'].str.contains('z플립4|플립4', na=False), 'keyword2'].value_counts()  # z플립4 검색어 제외한 코어커런스 데이터
pd.DataFrame(temp).head(100)    # 상위 100개

In [ ]:
# 연령대 정제
data_co['age2'] = data_co['age'].apply(lambda x: '-18' if x in ['12','13-18']
                                       else '19-29' if x in ['19-24','25-29']
                                       else '30-39' if x in ['30-34','35-39']
                                       else '40-49' if x in ['40-44','45-49']
                                       else '50-')

### 1. 성-연령별 코어커런스 검색어

In [ ]:
"""20대 여성의 코어커런스 검색어"""
temp = data_co.loc[(~data_co['keyword2'].str.contains('z플립4', na=False))&
                   (data_co['gender']=='f')&
                   (data_co['age2']=='19-29'), 'keyword2'].value_counts()
pd.DataFrame(temp).head(50)

In [ ]:
"""30대 남성의 코어커런스 검색어"""
temp = data_co.loc[(~data_co['keyword2'].str.contains('z플립4', na=False))&
                   (data_co['gender']=='m')&
                   (data_co['age2']=='30-39'), 'keyword2'].value_counts()
pd.DataFrame(temp).head(50)

### 2.  z플립4 포함 코어커런스 검색어
- 함께 검색한 검색어 + 클릭한 area 집계

In [ ]:
data_co['count'] = pd.to_numeric(data_co['count'], errors='coerce')
temp = data_co.groupby(by=['keyword1','area1','keyword2',' area2'])['count'].sum().reset_index()

# keyword1, keyword2의 모든 조합에 대해 count를 sum
temp.drop(temp[temp['count'] <= 100].index, inplace=True)   # 용량 다운사이징 >> 100번 이하의 조합은 제거

In [ ]:
# z플립4 포함된 코어커런스 검색어 상위 200개
print(temp.shape)
temp.sort_values(by='count', ascending=False).head(200)

In [ ]:
# print된 상위 200개 keyword1 각각에 대해 상위 100개의 keyword2 도출
temp2 = temp.sort_values(by='count', ascending=False).head(200)
print(set(temp2['keyword1']))
temp[temp['keyword1'].isin(set(temp2['keyword1']))]\
                    .groupby('keyword1')\
                    .apply(lambda x: x.sort_values(by='count', ascending=False)[:-5])